In [1]:
# Transformers installation
! pip install transformers datasets evaluate seqeval accelerate
# To install from source instead of the last release, comment the command above and uncomment the following one.
# ! pip install git+https://github.com/huggingface/transformers.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 52.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 44.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 102.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.7 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=1

In [2]:
pretrained_model = "bert-base-cased"

In [3]:
#Loading Data
import json
with open("main.json", encoding='utf-8') as fp:
    dataset = json.load(fp);

'''for i in range(len(dataset)):
    for j in range(len(dataset[i]['tokens'])):
        dataset[i]['tokens'][j] = dataset[i]['tokens'][j].upper()'''
dataset[0]

{'id': '0',
 'ner_tags': ['B-no', 'B-method', 'B-no', 'B-method', 'B-class', 'B-no'],
 'tokens': ['3.1.1', 'Start', 'and', 'Stop', 'Diagnostics', 'Use Case']}

In [4]:

label2id = {
    "O":0,
    "B-no":1,
    "I-no":2,
    "B-class":3,
    "I-class":4,
    "B-method":5,
    "I-method":6,
    "B-attribute":7,
    "I-attribute":8,
    "B-association":9,
    "I-association":10,
    "B-generalization":11,
    "I-generalization":12,
}

id2label = {
    0:"O",
    1:"B-no",
    2:"I-no",
    3:"B-class",
    4:"I-class",
    5:"B-method",
    6:"I-method",
    7:"B-attribute",
    8:"I-attribute",
    9:"B-association",
    10:"I-association",
    11:"B-generalization",
    12:"I-generalization",
}

In [5]:
for i in range(len(dataset)):
    dataset[i]["ner_tags"] = [label2id[j] for j in dataset[i]["ner_tags"]]

dataset[0]

{'id': '0',
 'ner_tags': [1, 5, 1, 5, 3, 1],
 'tokens': ['3.1.1', 'Start', 'and', 'Stop', 'Diagnostics', 'Use Case']}

In [6]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(dataset,test_size=0.15)
print(len(train_set),len(test_set))

3725 658


In [7]:
from datasets import IterableDataset

def gen_train():
    for i in train_set:
        yield i

def gen_test():
    for i in test_set:
        yield i

train_dataset = IterableDataset.from_generator(gen_train)
test_dataset = IterableDataset.from_generator(gen_test)

In [8]:
label_list = list(label2id.keys())
label_list

['O',
 'B-no',
 'I-no',
 'B-class',
 'I-class',
 'B-method',
 'I-method',
 'B-attribute',
 'I-attribute',
 'B-association',
 'I-association',
 'B-generalization',
 'I-generalization']

In [9]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(pretrained_model)

In [10]:
# example = wnut["train"][0]
example = dataset[0]
print(example)
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
tokens

{'id': '0', 'ner_tags': [1, 5, 1, 5, 3, 1], 'tokens': ['3.1.1', 'Start', 'and', 'Stop', 'Diagnostics', 'Use Case']}


['[CLS]',
 '3',
 '.',
 '1',
 '.',
 '1',
 'Start',
 'and',
 'Stop',
 'Di',
 '##ag',
 '##nos',
 '##tics',
 'Use',
 'Case',
 '[SEP]']

In [11]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [12]:
# tokenized_dataset = list(map(tokenize_and_align_labels, dataset))
tokenized_train_dataset = train_dataset.map(tokenize_and_align_labels, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_and_align_labels, batched=True)

In [13]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [14]:
import evaluate

seqeval = evaluate.load("seqeval")

In [15]:
import numpy as np

labels = [label_list[i] for i in example[f"ner_tags"]]


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [40]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(
    pretrained_model, num_labels=13, id2label=id2label, label2id=label2id)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
training_args = TrainingArguments(
    output_dir="model",
    learning_rate=0.01,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    max_steps = 500,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)


In [27]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
0,No log,1.422229,0.535327
1,No log,1.411875,0.535327
2,1.446400,1.399999,0.535327


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are i

TrainOutput(global_step=500, training_loss=1.4463948974609375, metrics={'train_runtime': 174.3274, 'train_samples_per_second': 45.891, 'train_steps_per_second': 2.868, 'total_flos': 356234958804756.0, 'train_loss': 1.4463948974609375, 'epoch': 2.07})

In [75]:
trainer.save_model("TokenClassifer")

In [76]:
text = "Start Diagnostics"

In [77]:
tokenizer_load = AutoTokenizer.from_pretrained("TokenClassifier")
model_load = AutoModelForTokenClassification.from_pretrained("TokenClassifier")

In [82]:
from transformers import pipeline

classifier = pipeline("ner", model=model_load, tokenizer = tokenizer_load)
classifier(text)

[]